In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data description

The dataset contains satellite images covering different landscapes like rural areas, urban areas, densely forested, mountainous terrain, small to large water bodies, agricultural areas, etc. covering the whole state of California. It is avaibable here : https://www.kaggle.com/crawford/deepsat-sat6?select=sat-6-full.mat.

The images are 28x28 pixels with 4 bands - red, green, blue and near infrared. We take 100.000 of such images for the train set and 50.000 for test set. Finally, the training and test labels are one-hot encoded 1x6 vectors for the 6 different classes of landscapes. 

In [24]:
%ls

In [25]:
train_data_path="../input/deepsat-sat6/X_train_sat6.csv"
train_label_path="../input/deepsat-sat6/y_train_sat6.csv"
test_data_path="../input/deepsat-sat6/X_test_sat6.csv"
test_lable_path="../input/deepsat-sat6/y_test_sat6.csv"

In [ ]:
train_data_path="archive/X_train_sat6.csv"
train_label_path="archive/y_train_sat6.csv"
test_data_path="archive/X_test_sat6.csv"
test_lable_path="archive/y_test_sat6.csv"

In [26]:
def data_read(data_path, nrows):
    data=pd.read_csv(data_path, header=None, nrows=nrows, dtype=np.uint8)
    data=data.values ## converting the data into numpy array
    return data

In [27]:
##Read training data
train_data=data_read(train_data_path, nrows=100000)
print("Train data shape:" + str(train_data.shape))

##Read training data labels
train_data_label=data_read(train_label_path,nrows=100000)
print("Train data label shape:" + str(train_data_label.shape))
print()

##Read test data
test_data=data_read(test_data_path, nrows=50000)
print("Test data shape:" + str(test_data.shape))


##Read test data labels
test_data_label=data_read(test_lable_path,nrows=50000)
print("Test data label shape:" + str(test_data_label.shape))

In [28]:
example = train_data[0]
example.shape

In [29]:
28**2 *4

In [30]:
reshaped_ex = example.reshape((28,28,4))[:,:,:3] #convert to rgb 
reshaped_ex.shape

In [31]:
from matplotlib import pyplot as plt 

In [32]:
plt.imshow(reshaped_ex)

In [33]:
ex_label = train_data_label[0]
ex_label

In [34]:
# Convert to RGB for now
train_data_reshaped = train_data.reshape(100000,28,28,4)[:,:,:,:3] / 255.
test_data_reshaped = test_data.reshape(50000,28,28,4)[:,:,:,:3] / 255.

# Strategy

Some models have already been designed for this specific dataset and problem and give good result. Our interest here will be  to compute different types of models and to study their robustnesses to small changes

## Resistance to adversarial attacks

Our goal is to measure the robustness of several models to adversarial attacks. 

In order to do this, we will use 3 different models: a simple convolutionnal net, a convolutionnal model using transfer learning and a model with computer vision features. And we will try 2 types of adversarial attacks: the **Fast Gradient Sign Method** (FGSM) attack and a **black box** attack. 

We will see later how to measure the robustness of a model to adversarial attacks.

## What are adversarial attacks ?

For context, there are many categories of adversarial attacks, each with a different goal and assumption of the attacker’s knowledge. However, in general the overarching goal is to add the least amount of perturbation to the input data to cause the desired misclassification. There are several kinds of assumptions of the attacker’s knowledge, two of which are: **white-box** and **black-box**. A white-box attack assumes the attacker has full knowledge and access to the model, including architecture, inputs, outputs, and weights. A black-box attack assumes the attacker only has access to the inputs and outputs of the model, and knows nothing about the underlying architecture or weights. There are also several types of goals, including **misclassification** and **source/target misclassification**. A goal of misclassification means the adversary only wants the output classification to be wrong but does not care what the new classification is. A source/target misclassification means the adversary wants to alter an image that is originally of a specific source class so that it is classified as a specific target class. In this notebook, we will focus on non targeted misclassification.

## Different types of adversarial attacks

Here, we're interested in two different attacks: the Fast Gradient Sign Method attack and a black box attack. 

# Different models in pytorch

We want to compare the resistance of different models to adversarial attacks. We test three different models : a simple convolutional network, a network vgg-based trained with transfert learning, and a model with no convolutional layers that uses only other computer vision features. 

In [35]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [36]:
class SatImgDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.transform = T.ToTensor()
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        x = self.transform(self.X[index])
        y = torch.FloatTensor(self.y[index])
        return {'x':x, 'y':y}
        
        

In [37]:
np.save("train_f.npy",train_data)
np.save("test_f.npy",test_data)

In [38]:
np.save("train_y.npy",train_data_label)
np.save("test_y.npy",test_data_label)

In [39]:
train_data_reshaped = train_data.reshape(100000,28,28,4)[:,:,:,:3] 
test_data_reshaped = test_data.reshape(50000,28,28,4)[:10000,:,:,:3] 

In [40]:
dataset_test = SatImgDataset(test_data_reshaped, test_data_label[:10000])
dataset_train = SatImgDataset(train_data_reshaped, train_data_label)

loader_train = DataLoader(dataset_train, 512, shuffle=True)
loader_test = DataLoader(dataset_test, 512, shuffle=False)

## Simple convolutional model

We computed a simple convolutional network that we trained from scratch on a part of the dataset (100000 images). The network is defined as follows:


*   Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3))
*   MaxPooling2D((2,2)
*   Conv2D(64, (3,3), activation='relu')
*   MaxPooling2D((2,2))
*   Flatten()
*   Dense(512, activation='relu')
*   Dense(6, activation='softmax')


with a cross-entropy loss and an Adam optimizer with learning rate $10e^-4$. This model will give us an accuracy of 96% after 100 epochs.

In [41]:
device = torch.device('cuda')
model = nn.Sequential(
    nn.Conv2d(3, 32, (3,3)),
    nn.ReLU(),
    nn.MaxPool2d((2,2)),
    nn.Conv2d(32, 64, (3,3)),
    nn.ReLU(),
    nn.MaxPool2d((2,2)),
    nn.Flatten(),
    nn.Linear(1600,512),
    nn.ReLU(),
    nn.Linear(512, 6),
    nn.Softmax()
).to(device)

In [42]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 30
criterion = nn.BCELoss()
for e in range(epochs):
    for batch in tqdm(loader_train):
        pred = model(batch['x'].to(device))
        loss = criterion( pred, batch['y'].to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

### Gradient based Attacks : FGSM

One of the first and most popular adversarial attacks to date is referred to as the Fast Gradient Sign Attack (FGSM) and is described by Goodfellow et. al. in *Explaining and Harnessing Adversarial Examples* (https://arxiv.org/abs/1412.6572). 

The attack is remarkably powerful, and yet intuitive. It is designed to attack neural networks by leveraging the way they learn, gradients. The idea is simple, rather than working to minimize the loss by adjusting the weights based on the backpropagated gradients, the attack adjusts the input data to maximize the loss based on the same backpropagated gradients. In other words, the attack uses the gradient of the loss w.r.t the input data, then adjusts the input data to maximize the loss. 


For instance, let $\mathbf{x}$ be the original input image, $y$ the ground truth label for $\mathbf{x}$, $\mathbf{\theta}$ the model parameters, and $J(\mathbf{\theta}, \mathbf{x}, y)$ the loss that is used to train the network. The attack backpropagates the gradient back to the input data to calculate $\nabla_{\mathbf{x}} J(\mathbf{\theta}, \mathbf{x}, y)$. 


Then, it adjusts the input data by a small step $\epsilon$ in the direction that will maximize the loss (i.e. $sign(\nabla_{\mathbf{x}} J(\mathbf{\theta}, \mathbf{x}, y))$). The resulting perturbed image, $x'$, is then misclassified by the target network. 

In [43]:
def fast_gradient_sign_method(model, imgs, labels, attack_params):
    # Determine prediction of the model
    inp_imgs = imgs.clone().requires_grad_()
    preds = model(inp_imgs.to(device))
    pred_logprob = torch.log(preds)
    #print(labels.dtype)
    # Calculate loss by NLL
    loss = nn.BCELoss()(preds, labels.to(device))
    loss.backward()
    # Update image to adversarial example as written above
    noise_grad = torch.sign(inp_imgs.grad.to(imgs.device))
    fake_imgs = imgs + attack_params['epsilon'] * noise_grad
    fake_imgs.detach_()
    return fake_imgs, noise_grad

In [44]:
def eval_model(model, dataloader, fake_generator=None, attack_params=None):
    y_pred = []
    y_true = []
    
    for i, batch in enumerate(dataloader):
        #print(batch['y'].shape)
        if fake_generator is not None:
            fake_imgs, _ = fake_generator(model, batch['x'],batch['y'], attack_params)
            with torch.no_grad():
                pred = model(fake_imgs.to(device))
        else:
            with torch.no_grad():
                pred = model(batch['x'].to(device))
        y_pred.append(pred.argmax(dim=1).cpu().numpy())
        y_true.append(batch['y'].argmax(dim=1).numpy())
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    return accuracy_score(y_true, y_pred)

In [45]:
eps = np.logspace(-2, 0, 10)
acc_attack = [eval_model(model, loader_test, fast_gradient_sign_method, {'epsilon': e}) for e in eps]


In [46]:
plt.figure()
plt.plot(eps, acc_attack)
plt.xscale('log')
plt.ylabel("accuracy")
plt.xlabel('epsilon')
plt.show()

In [47]:
def predict_one_image(img_data, y):
    with torch.no_grad():
        pred = model(T.ToTensor()(img_data).unsqueeze(0).to(device))
    true_label = y.argmax()
    pred_label = pred[0].argmax()
    print(pred)
    plt.imshow(img_data)
    print("True class: {}, predicted class: {}".format(true_label, pred_label))

In [48]:
attack_params = {'epsilon':0.03}

img_data = test_data[223].reshape((28,28,4))[:,:,:3]
labels = torch.FloatTensor(test_data_label[223]).unsqueeze(0)

f_img, noise_grad = fast_gradient_sign_method(model, T.ToTensor()(img_data).unsqueeze(0), labels, attack_params)
fake_img = f_img.squeeze().numpy().transpose(1,2,0)



In [49]:
predict_one_image(img_data, labels)


In [50]:
predict_one_image(fake_img, labels)


### Black box attack : SIMBA

It will be particularly useful for the model with computer vision features, for which we do not have access to gradients, thus we can't use white box attacks. Indeed, black-box attacks require only queries to the target model that may return complete or partial information. The algorithm we use (described in https://arxiv.org/abs/1905.07121) uses the following simple iterative principle: we randomly sample a vector from a predefined orthonormal basis and either add or subtract it to the target image.

We assume we have some image $\mathbf{x}$ which a black-box neural network, $h$, classifies $h(\mathbf{x}) = y$ with predicted confidence or output probability $p_h(y | \mathbf{x})$. The goal is to find a small perturbation $\delta$ such that the prediction $h(\mathbf{x} + \delta) = y$. 

Although gradient information is absent in the black-box setting, we argue that the presence of output probabilities can serve as a strong proxy to guide the search for adversarial images. The intuition behind the method is simple: for any direction $\mathbf{q}$ and some step size $\epsilon$, one of $\mathbf{x} + \epsilon \mathbf{q}$ or $\mathbf{x} − \epsilon \mathbf{q}$ is likely to decrease $p_h(y | \mathbf{x})$. 

We therefore repeatedly pick random directions $\mathbf{q}$ and either add or subtract them. To minimize the number of queries to $h(·)$ we always first try adding $\mathbf{q}$. If this decreases the probability $p_h(y | \mathbf{x})$ we take the step, otherwise we try subtracting $\mathbf{q}$. This procedure requires between $1.4$ and $1.5$ queries per update on average (depending on the data set and target model). 

The method – Simple Black-box Attack (SimBA) – takes as input the target image label pair $(\mathbf{x},y)$, a set of orthonormal candidate vectors $\mathcal{Q}$ and a step-size $\epsilon > 0$. For simplicity we pick $\mathbf{q} \in \mathcal{Q}$ uniformly at random. To guarantee maximum query efficiency, we ensure that no two directions cancel each other out and diminish progress, or amplify each other and increase the norm of $\delta$ disproportionately. For this reason we pick $\mathbf{q}$ without replacement and restrict all vectors in $\mathcal{Q}$ to be orthonormal. The only hyper-parameters of SimBA are the set of orthogonal search vectors $\mathcal{Q}$ and the step size $\epsilon$.


The parameters $\epsilon$ will help us quantify how robust our models are. The bigger $\epsilon$ needs to be in order to get images misclassified, the more robust our models are. 

In [51]:
def get_probs(model, x, y):
    output = model(x)
    probs = torch.gather(output, 1, y.unsqueeze(1)).squeeze()
    return probs

def simba_single(model, img, y, num_iters=1000, epsilon=0.05):
    x = img.clone()
    n_dims = x.view(x.size(0), -1).size(1)
    perm = torch.randperm(n_dims)
    #print(x.view(x.size(1), -1).shape)
    last_prob = get_probs(model, x, y)
    for i in range(num_iters):
        diff = torch.zeros(x.shape[0], n_dims).to(device)
        diff[:, perm[i]] = epsilon
        with torch.no_grad():
            left_prob = get_probs(model, (x - diff.view(x.size())).clamp(0, 1), y)
            right_prob = get_probs(model, (x + diff.view(x.size())).clamp(0, 1), y)
        
        left_mask = left_prob < last_prob
        right_mask = right_prob < last_prob
        
        
        x[left_mask] = (x[left_mask] - diff[left_mask].view(x[left_mask].size())).clamp(0, 1)
        x[right_mask] = (x[right_mask] + diff[right_mask].view(x[right_mask].size())).clamp(0, 1)
        
        last_prob[left_mask] = left_prob[left_mask]
        last_prob[right_mask] = right_prob[right_mask]


    return x.squeeze()

In [52]:

eps = np.logspace(-2, 0, 10)
acc_attack = []
fake=True
for e in eps:
    y_pred = []
    y_true = []
    for i, batch in enumerate(tqdm(loader_test)):
        #print(batch['y'].shape)
        if fake:
            fake_imgs = simba_single(model, 
                                     batch['x'].to(device) ,
                                     batch['y'].argmax(dim=1).to(device),
                                    epsilon=e)
            with torch.no_grad():
                pred = model(fake_imgs.to(device))
        else:
            with torch.no_grad():
                pred = model(batch['x'].to(device))
        y_pred.append(pred.argmax(dim=1).cpu().numpy())
        y_true.append(batch['y'].argmax(dim=1).numpy())
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    acc_attack.append(accuracy_score(y_true, y_pred))

In [53]:
plt.figure()
plt.plot(eps, acc_attack)
plt.xscale('log')
plt.ylabel("accuracy")
plt.xlabel('epsilon')
plt.savefig("cnn_simba.png")
plt.show()

In [54]:
img_data = test_data[12].reshape((28,28,4))[:,:,:3]
labels = torch.FloatTensor(test_data_label[12]).unsqueeze(0).to(device)

f_img = simba_single(model, T.ToTensor()(img_data).unsqueeze(0).to(device), labels.argmax(1), num_iters=500)
fake_img = f_img.squeeze().cpu().numpy().transpose(1,2,0)



In [55]:
predict_one_image(fake_img, labels)

In [56]:
predict_one_image(img_data, labels)

## Model with transfert learning

We have a big dataset with images to classify, so we try to do tranfert learning to benefit from pre-training of models on other datasets.

The main problems concerning transfert learning is that:


*   The datasets on which pretrained models are trained are very different from satellite images, so the features that are learned may not be relevant for our problem. This is why we looked for models trained on satellite images. We found some pretrained models such as BigEarthNet, but the minimal input size was ten times the size of our images, so we did not use these models.

*   The images at our disposal are very small (28x28), whereas pretrained models take inputs of at least 32x32, since it is the minimal dimension of the network. We solved this problem by using the resize function from open-cv that allows to upscale an image.

We used VGG-16 where we replaced the last layer by a dense layer of output size 6 (the number of classes). We froze all VGG layers except the final one and we trained the model. 

We got a score close to the score from our first CNN, but this model was ten times longer to train. The score was not increased.

In [57]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from cv2 import resize

In [58]:
train_data_reshaped_big = []
test_data_reshaped_big = []

for i in range(len(train_data_reshaped[:50000])):
    train_data_reshaped_big.append(resize(train_data_reshaped[i], (32,32)))
    
for i in range(len(test_data_reshaped[:25000])):
    test_data_reshaped_big.append(resize(test_data_reshaped[i], (32,32)))
    
train_data_reshaped_big, test_data_reshaped_big = np.array(train_data_reshaped_big), np.array(test_data_reshaped_big)

In [62]:
train_data_reshaped_big.shape

In [63]:
dataset_test = SatImgDataset(test_data_reshaped_big, test_data_label[:50000])
dataset_train = SatImgDataset(train_data_reshaped_big, train_data_label[:25000])

loader_train = DataLoader(dataset_train, 512, shuffle=True)
loader_test = DataLoader(dataset_test, 512, shuffle=False)

In [64]:
model_pretrained = models.vgg16(pretrained=True)

for param in model_pretrained.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
number_features = model_pretrained.classifier[6].in_features
features = list(model_pretrained.classifier.children())[:-1] # Remove last layer
features.extend([torch.nn.Linear(number_features, 6)])
model_pretrained.classifier = torch.nn.Sequential(*features)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer_ft = optim.SGD(model_pretrained.parameters(), lr=0.001, momentum=0.9)

model_pretrained = model_pretrained.to(device)

In [65]:
epochs = 30
for e in range(epochs):
    for batch in tqdm(loader_train):
        pred = model_pretrained(batch['x'].float().to(device))
        loss = criterion( pred, batch['y'].to(device))
        optimizer_ft.zero_grad()
        loss.backward()
        optimizer_ft.step()

In [ ]:
print("Accuracy : ",eval_model(model_pretrained, loader_test))

In [ ]:
eps = np.logspace(-3, -1, 10)
acc_attack = [eval_model(model_pretrained, loader_test, fast_gradient_sign_method, {'epsilon': e}) for e in eps]

In [ ]:
plt.figure()
plt.plot(eps, acc_attack)
plt.xscale('log')
plt.ylabel("accuracy")
plt.xlabel('epsilon')
plt.show()

## Model with computer vision features

We did not compute this model, it is public on the Kaggle page of the Deepsat dataset, it was created by Rahul KUMAR.

The particularities of this model is that it does not use convolutions to extract features from the images, but computer vision features. We detail below these features:



*   texture features
*   NIR : near infrared channel
*   HSV features
*   NDVI features
*   ARVI

This gives 17 features in total per image.

Then, these vectors of features are passed through a network composed of these three layers:
* Dense(50), Activation("relu"), Dropout(0.2))
* Dense(50), Activation("relu"), Dropout(0.2)
* Dense(6, activation="softmax")

After 500 epochs, the score is 99.38%.

In [66]:
import numpy as np
import time
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px
from skimage import color
!pip install mahotas
import mahotas as mt
from skimage.color import rgb2gray
import warnings
warnings.filterwarnings("ignore")

In [67]:
def feature_extractor(data):
    
        tex_feature=[]
        hsv_feature=[]
        ndvi_feature=[]
        arvi_feature=[]

        #for df_chunk in tqdm(pd.read_csv(input_image_file ,header=None,chunksize = 5000)):
            
        #df_chunk=df_chunk.astype("int32")
        #data=df_chunk.values


        ################data for HSV and Texture feature##############
        img=data.reshape(-1,28,28,4)[:,:,:,:3]
        #############################################################

        ######################Data for NDVI and ARVI#################

        NIR=data.reshape(-1,28,28,4)[:,:,:,3]
        Red=data.reshape(-1,28,28,4)[:,:,:,2]
        Blue=data.reshape(-1,28,28,4)[:,:,:,0]
        #############################################################

        for i in range(len(data)):

            #######Texture_feature####################################
            textures = mt.features.haralick(img[i])
            ht_mean= textures.mean(axis=0)
            tex_feature.append(ht_mean)
            ##########################################################

            #######hsv_feature#########################################
            img_hsv = color.rgb2hsv(img[i]) # Image into HSV colorspace
            h = img_hsv[:,:,0] # Hue
            s = img_hsv[:,:,1] # Saturation
            v = img_hsv[:,:,2] # Value aka Lightness
            hsv_feature.append((h.mean(),s.mean(),v.mean()))
            ###########################################################

            ##########Calculation of NDVI Feature######################
            NDVI=(NIR[i]-Red[i])/(NIR[i]+Red[i])
            ndvi_feature.append(NDVI.mean())
            ############################################################

            ###################Calculation of ARVI#####################
            a_1=NIR[i] -(2*Red[i]-Blue[i])
            a_2=NIR[i] +(2*Red[i]+Blue[i])
            arvi=a_1/a_2
            arvi_feature.append(arvi.mean())
            #######################################################

        features=[]
        for i in range(len(tex_feature)):
            h_stack=np.hstack((tex_feature[i], hsv_feature[i], ndvi_feature[i], arvi_feature[i]))
            features.append(h_stack)

        return np.array(features)

In [ ]:
train_data_features=feature_extractor(train_data[:50000])


In [ ]:
np.save( "train_cv.npy",train_data_features)

In [ ]:
np.save("test_cv.npy", test_data_features)

In [ ]:
test_data_features=feature_extractor(test_data[:20000])

In [ ]:
if train_data_features.shape[1]==18:
    train_data_features_r = train_data_features[:,:-2]
    test_data_features_r = test_data_features[:,:-2]
sc=StandardScaler()
#fit the training data
fit=sc.fit(train_data_features)

##transform the train and test data
train_data_stn=fit.transform(train_data_features)
test_data_stn=fit.transform(test_data_features)

In [ ]:
mlp_model = nn.Sequential(
    nn.Linear(16, 50),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(50,6),
    nn.Softmax()
).to(device)

In [ ]:
class SatImgDatasetCV(Dataset):
    def __init__(self, X, y, images):
        self.X = X
        self.y = y
        self.images = images
        self.transform = T.ToTensor()
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        img = self.transform(self.images[index])
        y = torch.FloatTensor(self.y[index])
        x = torch.FloatTensor(self.X[index])
        return {'x':x, 'y':y, 'img': img}

In [ ]:
train_data_reshaped = train_data.reshape(100000,28,28,4)[:,:,:,:3] 
test_data_reshaped = test_data.reshape(50000,28,28,4)[:,:,:,:3] 

In [ ]:
dataset_test_f = SatImgDatasetCV(test_data_stn, test_data_label[:20000], test_data.reshape(-1,28,28,4))
dataset_train_f = SatImgDatasetCV(train_data_stn, train_data_label[:50000],train_data.reshape(-1,28,28,4))

loader_train_f = DataLoader(dataset_train_f, 5000, shuffle=True)
loader_test_f = DataLoader(dataset_test_f, 512, shuffle=False)

In [ ]:
mlp_model.train()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=1e-3)
epochs = 50
criterion = nn.BCELoss()
for e in range(epochs):
    for batch in tqdm(loader_train_f):
        pred = mlp_model(batch['x'].to(device))
        loss = criterion( pred, batch['y'].to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
mlp_model.eval()
pred_test = []
y_test = []
with torch.no_grad():
    for batch in tqdm(loader_test_f):
        pred = mlp_model(batch['x'].to(device))
        pred_test.append(pred.argmax(1).cpu().squeeze().numpy())
        y_test.append(batch['y'].argmax(1).cpu().squeeze().numpy())
        
pred_test = np.concatenate(pred_test)
y_test = np.concatenate(y_test)
print("accuracy_score = {}".format(accuracy_score(y_test, pred_test)))

In [ ]:
def get_probs(model, x, y):
    output = model(x)
   # print(output)
    probs = torch.gather(output, 1, y.unsqueeze(1)).squeeze()
    return probs

def simba_single_mlp(model, img, y, num_iters=1000, epsilon=0.05):
    x = img.clone()
    
    n_dims = x.view(x.size(0), -1).size(1)
    perm = torch.randperm(n_dims)
    
    x_f = (x.permute(0,2,3,1).reshape(x.size(0),-1).cpu().detach().numpy()*255).astype(np.uint8)
   # print(x_f.shape)
    x_f = fit.transform(feature_extractor(x_f)[:,:-2])

    last_prob = get_probs(model, torch.FloatTensor(x_f).to(device), y)
    for i in tqdm(range(num_iters)):
        
        diff = torch.zeros(x.shape[0], n_dims).to(device)
        diff[:, perm[i]] = epsilon
        
        with torch.no_grad():

            x_left = (x - diff.view(x.size())).clamp(0, 1)
            x_right = (x + diff.view(x.size())).clamp(0, 1)

            x_f = feature_extractor((x.permute(0,2,3,1).reshape(x.size(0),-1).cpu().detach().numpy()*255).astype(np.uint8))[:,:-2]
            x_l_f = feature_extractor((x_left.permute(0,2,3,1).reshape(x.size(0),-1).cpu().detach().numpy()*255).astype(np.uint8))[:,:-2]
            x_r_f = feature_extractor((x_right.permute(0,2,3,1).reshape(x.size(0),-1).cpu().detach().numpy()*255).astype(np.uint8))[:,:-2]
            left_prob = get_probs(model, torch.FloatTensor(fit.transform(x_l_f)).to(device), y)
            right_prob = get_probs(model, torch.FloatTensor(fit.transform(x_r_f)).to(device), y)

        left_mask = left_prob < last_prob
        right_mask = right_prob < last_prob
        
        x[left_mask] = x_left[left_mask]
    
        x[right_mask] = x_right[right_mask]
        
        last_prob[left_mask] = left_prob[left_mask]
        last_prob[right_mask] = right_prob[right_mask]

    return x

In [ ]:
img_data = test_data[12].reshape((28,28,4))
labels = torch.FloatTensor(test_data_label[12]).unsqueeze(0).to(device)

f_img = simba_single_mlp(mlp_model, T.ToTensor()(img_data).unsqueeze(0).to(device), labels.argmax(1), num_iters=70, epsilon=0.3)
fake_img = f_img.squeeze().cpu().numpy().transpose(1,2,0)

In [ ]:
def predict_one_image_mlp(img_data, y):
    #print("ok", img_data.reshape(1,-1).mean())
    with torch.no_grad():
     #   print(feature_extractor(img_data.reshape(1,-1))[:,:-2])
        x_f = fit.transform(feature_extractor(img_data.reshape(1,-1))[:,:-2])
      #  print(x_f)
        pred = mlp_model(torch.FloatTensor(x_f).to(device))
    true_label = y.argmax()
    pred_label = pred[0].argmax()
    print(pred)
    plt.imshow(img_data[:,:,:3])
    print("True class: {}, predicted class: {}".format(true_label, pred_label))

In [ ]:
predict_one_image_mlp(img_data, labels)

In [ ]:
predict_one_image_mlp((fake_img * 255).astype(np.uint8), labels)

In [ ]:
eps = [0.05, 0.2, 0.5]
acc_attack = []

for e in eps:
    for i in range(20):
        y_pred = []
        y_true = []
        #print(batch['y'].shape)
        f_img = simba_single_mlp(mlp_model, dataset_test_f[i]['img'].unsqueeze(0).to(device), 
                                 dataset_test_f[i]['y'].argmax().unsqueeze(0).to(device), 
                                 num_iters=500,
                                 epsilon=0.3)
        fake_img = (f_img.squeeze().cpu().numpy().transpose(1,2,0) * 255).astype(np.uint8)
        x_f = fit.transform(feature_extractor(fake_img.reshape(1,-1))[:,:-2])
        with torch.no_grad():
            pred = mlp_model(torch.FloatTensor(x_f).to(device))


        y_pred.append(pred.argmax().cpu().numpy())
        y_true.append(dataset_test_f[i]['y'].argmax().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    acc_attack.append(accuracy_score(y_true, y_pred))

In [ ]:
plt.figure()
plt.plot(eps, acc_attack)
plt.xscale('log')
plt.ylabel("accuracy")
plt.xlabel('epsilon')
plt.savefig("cvf_simba.png")
plt.show()

# Annex : Keras models, training and scores

## Simple CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import optimizers

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', 
             optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
history = model.fit(train_data_reshaped, train_data_label, validation_split=0.20, batch_size=512, epochs=100, callbacks=[es])

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, 'bo', label='Entraînement')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation')

plt.title('Perte pdt l\'entraînement et la validation')
plt.xlabel("Nombre d'époques")
plt.ylabel('Perte')
plt.legend()
plt.show()

In [ ]:
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc_values, 'bo', label='Entraînement')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation')

plt.title('Exactitude pdt l\'entraînement et la validation')
plt.xlabel("Nombre d'époques")
plt.ylabel('Exactitude de prediction')
plt.legend()
plt.show()

In [ ]:
test_accuracy = model.evaluate(test_data_reshaped, test_data_label)[1]
test_accuracy

In [ ]:
from tensorflow.math import confusion_matrix

In [ ]:
test_data_label_pred = model.predict(test_data_reshaped)

In [ ]:
confusion_matrix(test_data_label.argmax(axis=1), test_data_label_pred.argmax(axis=1))

## Transfert learning with VGG-16

In [ ]:
#from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from cv2 import resize

In [ ]:
train_data_reshaped_big = []
test_data_reshaped_big = []

for i in range(len(train_data_reshaped)):
    train_data_reshaped_big.append(resize(train_data_reshaped[i], (32,32)))
    
for i in range(len(test_data_reshaped)):
    test_data_reshaped_big.append(resize(test_data_reshaped[i], (32,32)))
    
train_data_reshaped_big, test_data_reshaped_big = np.array(train_data_reshaped_big), np.array(test_data_reshaped_big)

In [ ]:
vgg_model = Sequential()

pretrained_model= VGG16(include_top=False,
                   input_shape=(32,32,3),
                   pooling='avg',classes=6,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

vgg_model.add(pretrained_model)

vgg_model.add(Flatten())
vgg_model.add(Dense(256, activation='relu'))
vgg_model.add(Dense(6, activation='softmax'))

In [ ]:
vgg_model.summary()

In [ ]:
vgg_model.compile(loss="categorical_crossentropy",
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

In [ ]:
history2 = vgg_model.fit(train_data_reshaped_big, train_data_label, validation_split=0.20, batch_size=512, epochs=3, callbacks=[es])